# Vacationpy Analysis

In [3]:
# import dependencies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [5]:
#importing weather data from weatherPy exported csv
weather_df = pd.read_csv("weather_data.csv")
weather_df


,City,country,Date,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind Speed
0,Mataura,NZ,1650139304,-46.1927,168.8643,276.75,96,68,1.22
1,Cidreira,BR,1650139304,-30.1811,-50.2056,292.73,68,100,3.42
2,Novy Urengoy,RU,1650139121,66.0833,76.6333,264.11,92,100,7.57
3,Dikson,RU,1650139305,73.5069,80.5464,261.80,94,100,7.26
4,Juan Lacaze,UY,1650139305,-34.4342,-57.4381,293.21,65,29,5.74
...,...,...,...,...,...,...,...,...,...
543,Ambanja,MG,1650139527,-13.6833,48.4500,297.97,90,19,1.33
544,Talara,PE,1650139528,-4.5772,-81.2719,300.57,54,0,6.69
545,Jijiga,ET,1650139528,9.3500,42.8000,293.92,65,96,3.04
546,Carauari,BR,1650139529,-4.8828,-66.8958,303.71,61,94,0.72


# Creating Humidity heatmap

In [8]:
#configure gmap with api key
gmaps.configure(api_key = g_key)


In [38]:
#storing latitude and logitude as locations
locations = weather_df[["Latitude", "Longitude"]].astype(float)

#getting humidity data
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()

max_humidity

100.0

In [40]:
#creating humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False,
                                 max_intensity = max_humidity, point_radius = 3)
fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))

# Creating new dataframe with narrowed down criteria


In [52]:
#limit max temp to a range of 70F (294 K) to 85F (302 K)
#limit windspeed < 12
#limit cloudiness <30
weather_pref = weather_df[(weather_df["Max_temp"]>=294) & (weather_df["Max_temp"]<=302)
                             & (weather_df["Wind Speed"]<=12)& (weather_df["Cloudiness"]<=20)]

weather_pref.shape


(54, 9)